In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# https://sites.google.com/a/chromium.org/chromedriver/ 에서 크롬용 selenium 드라이버를 다운받는다.
# 아래에 chromedriver 의 절대경로를 넣는다.
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)

# naver에 접근한다.
driver.get('https://www.naver.com/')

# 마녀를 검색어로 입력
driver.find_element_by_id('query').send_keys('마녀')
driver.find_element_by_id("query").send_keys(Keys.ENTER)

# 뉴스 탭으로 전환
driver.find_element_by_class_name('lnb4').click()

## 검색 결과 세분화
1. 네이버 검색페이지에서 볼 수 있는 결과는 최대 4,000개이기 때문에 내가 입력한 검색결과가 4,000개보다 많은 경우 검색 조건을 더 세밀하게 주어서 4,000개 이하로 줄여야 한다.

In [ ]:
import random 
from time import sleep
import pandas as pd

def find_article_links():
    result = []
    # 서브기사가 네이버 뉴스인 경우
    driver.find_elements_by_xpath("(//span[@class='txt_sinfo']/a)")
    for elem in driver.find_elements_by_xpath("(//span[@class='txt_sinfo']/a)"):
#         print(elem.text, elem.get_attribute('href'))
        result.append({
            'url': elem.get_attribute('href')
        })
        
    # 메인기사가 네이버 뉴스인 경우
    for elem in driver.find_elements_by_xpath("(//dd[@class='txt_inline']/a)"):
#         print(elem.text, elem.get_attribute('href'))
        result.append({
            'url': elem.get_attribute('href')
        })


    print(len(result))
    return pd.DataFrame(result)


# 검색페이지 번호를 저장하기 위한 목록
traversed = ['1']
traversed_size = len(traversed)
# 각 검색페이지에서 찾아낸 네이버뉴스 링크를 갖고 있는 목록
# list(DataFrame) 형태
article_df_list = []

# 현재 보고 있는 검색 페이지, 1페이지부터 시작
page_idx = '1'


while True:
    '''
    1. 현재 페이지에서 뉴스글 링크를 뽑아냄.
    '''
    article_df = find_article_links()
    article_df_list.append(article_df)
    
    sleep(1 + random.random() * 6)
    '''
    2. paging 링크들을 찾아냄.
    '''
    paging = driver.find_elements_by_xpath("(//div[@class='paging']/a)")
    print(len(paging))
    
    '''
    읽지 않은 검색페이지를 클릭
    '''
    for idx, elem in enumerate(paging):
        if elem.text in ['이전페이지', '다음페이지']:
            continue
        if elem.text in traversed:
            continue
        traversed.append(elem.text)
        print(elem.text, elem.get_attribute('href'))
        page_idx = elem.text
        paging[idx].click()
        break
    if traversed_size == len(traversed):
        break
    traversed_size = len(traversed)
    continue

In [ ]:
# 결과를 하나의 csv 파일로 저장
pd.concat(article_df_list).reset_index().to_csv('마녀검색.csv')